<a href="https://colab.research.google.com/github/Mostofa-Najmus-Sakib/NLP-works/blob/main/Assisgnment_6/A6_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment, you will use a pre-trained convnet to produce features for a classifier that can detect a single object type. This notebook has some code to help you get started. 

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My\ Drive/Colab\ Notebooks/NLP_2021/

/content/drive/My Drive/Colab Notebooks/NLP_2021


In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
!jupyter kernelspec list

Available kernels:
  ir         /usr/local/share/jupyter/kernels/ir
  python2    /usr/local/share/jupyter/kernels/python2
  python3    /usr/local/share/jupyter/kernels/python3


In [ ]:
import pandas as pd
import cv2
import os
from os import listdir
from os.path import isfile, join
import os.path as osp
from tqdm import tqdm_notebook as tqdm

In [ ]:
import requests
from imutils import paths
from google_images_download import google_images_download
img_folder = 'downloads'

### Gather positive examples

Pick a word. For example, "red" or "santa" or "horse". 

Now you will need to find "positive" image examples of that word. For example, if you chose "red" as your word, you will need to find images of red things. You are free to use Google Image search or something similar. File types shouldn't matter, but try to stick with .png and .jpg files.

You'll need at least 100 positive example images. Put them in the folder called `pos`. 

***I have downloaded 200 images for colur red using a extionsion of google chrome***

### Gather negative examples

Now you need to think about negative examples; i.e., things that are *not* examples of your word. You can either just find random images, or look for specific negative examples. For example, if you chose the word "red" then it might work best if you find negative examples that are other colors, especially colors close to red. 

You'll need at least 200 negative example images. Put them in the folder called `neg`. 

***I have downloaded 200 images for colur pink (close to red) using a extionsion of google chrome***

## 1.) Run the following cell

* This imports needed Keras libraries
* Then, it gets the trained VGG19 imagenet model
* Then, it prints out the names of all the layers in that model

In [ ]:
import numpy as np
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

base_model = VGG19(weights='imagenet',include_top=True)
xs,ys=224,224

for layer in base_model.layers:
    print(layer.name)

input_22
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool
flatten
fc1
fc2
predictions


### 2.) Determine your output layer

- try `predictions` first

In [ ]:
layer = 'predictions'

model = Model(inputs=base_model.input, outputs=base_model.get_layer(layer).output)

### Run the following cell

- These functions are to help you perform transfer learning

In [ ]:
def get_image(img_path, xs,ys):
    x = image.load_img(img_path, target_size=(xs, ys))
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    return x

def get_img_features(model, img):
    img = preprocess_input(img)
    yhat = model.predict(img)
    return yhat

def get_image_features(word):
    files = [f for f in listdir(word)] # grab all of the images in the folder
    image_vectors = []
    for f in tqdm(files):
        img = get_image(osp.join(word, f), xs, ys) 
        x_feats = get_img_features(model, img).flatten() # get features for each image
        image_vectors.append(x_feats) 
    return np.array(image_vectors)

## 3.) Evaluate a classifier for your `word`

* Using the positive and negative output from `model`, train a classifier (it can be a linear classifier from scikit-learn, if you'd like, but I would recommend the Keras Dense network we built for the previous assignment). 
* You'll need to split your data into Train and Test (I would recommend using half of the data for training, half for testing; you may opt for downloading more positive and negative examples)
* your classifier can be any scikit classifier, but you can also use a neural network of some kind

In [ ]:
pos_images = get_image_features('pos_red_3') # get positive image vectors
neg_images = get_image_features('neg_pink_3') # get negative image vectors

### Prepare the data. Split to train/test sets

In [ ]:
num_test = 50
X_train = np.concatenate((pos_images[num_test:], neg_images[num_test:]), axis=0)
X_test = np.concatenate((pos_images[:num_test], neg_images[:num_test]))

y_train = np.array([1] * (200 - num_test) + [0] * (200 - num_test))
y_test = np.array([1] * (num_test) + [0] * (num_test))

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((300, 1000), (300,), (100, 1000), (100,))

### Define model, train

In [ ]:
from sklearn.linear_model import LogisticRegression

clfr = LogisticRegression()
clfr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Evaluate

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(clfr.predict(X_test), y_test)

0.87

### 4.) Try a Different `base_model` 

* Repeat steps 3 and 4 above, only this time use a [pre-trained model other than VGG19](https://keras.io/applications/)
* (Answer in a markdown cell): Which model+layer works the best for this data? Why do you think that is?
* What makes for good positive examples? What makes for good negative examples? Why does the choice of negative examples matter?

### Lets try base model (VGG16) and layer flatten




In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

base_model = VGG16(weights='imagenet')

for layer in base_model.layers:
    print(layer.name)

input_13
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool
flatten
fc1
fc2
predictions


In [ ]:
layer = 'flatten'

model = Model(inputs=base_model.input, outputs=base_model.get_layer(layer).output)

In [ ]:
num_test = 50
X_train = np.concatenate((pos_images[num_test:], neg_images[num_test:]), axis=0)
X_test = np.concatenate((pos_images[:num_test], neg_images[:num_test]))

y_train = np.array([1] * (200 - num_test) + [0] * (200 - num_test))
y_test = np.array([1] * (num_test) + [0] * (num_test))

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((300, 1000), (300,), (100, 1000), (100,))

In [ ]:
from sklearn.linear_model import LogisticRegression

clfr = LogisticRegression()
clfr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(clfr.predict(X_test), y_test)

0.87

## For the same model layer = predictions

In [ ]:
layer = 'predictions'


model = Model(inputs=base_model.input, outputs=base_model.get_layer(layer).output)

# Prepare the data. Split to train/test sets
num_test = 50
X_train = np.concatenate((pos_images[num_test:], neg_images[num_test:]), axis=0)
X_test = np.concatenate((pos_images[:num_test], neg_images[:num_test]))

y_train = np.array([1] * (200 - num_test) + [0] * (200 - num_test))
y_test = np.array([1] * (num_test) + [0] * (num_test))

# X_train.shape, y_train.shape, X_test.shape, y_test.shape

# Define the model and train
from sklearn.linear_model import LogisticRegression

clfr = LogisticRegression()
clfr.fit(X_train, y_train)

# Evaluate
accuracy_score(clfr.predict(X_test), y_test)

0.87

## Lets try base model (DenseNet121) and layer predictions

In [ ]:
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input

base_model = DenseNet121(weights='imagenet')

for layer in base_model.layers:
    print(layer.name)

input_19
zero_padding2d_12
conv1/conv
conv1/bn
conv1/relu
zero_padding2d_13
pool1
conv2_block1_0_bn
conv2_block1_0_relu
conv2_block1_1_conv
conv2_block1_1_bn
conv2_block1_1_relu
conv2_block1_2_conv
conv2_block1_concat
conv2_block2_0_bn
conv2_block2_0_relu
conv2_block2_1_conv
conv2_block2_1_bn
conv2_block2_1_relu
conv2_block2_2_conv
conv2_block2_concat
conv2_block3_0_bn
conv2_block3_0_relu
conv2_block3_1_conv
conv2_block3_1_bn
conv2_block3_1_relu
conv2_block3_2_conv
conv2_block3_concat
conv2_block4_0_bn
conv2_block4_0_relu
conv2_block4_1_conv
conv2_block4_1_bn
conv2_block4_1_relu
conv2_block4_2_conv
conv2_block4_concat
conv2_block5_0_bn
conv2_block5_0_relu
conv2_block5_1_conv
conv2_block5_1_bn
conv2_block5_1_relu
conv2_block5_2_conv
conv2_block5_concat
conv2_block6_0_bn
conv2_block6_0_relu
conv2_block6_1_conv
conv2_block6_1_bn
conv2_block6_1_relu
conv2_block6_2_conv
conv2_block6_concat
pool2_bn
pool2_relu
pool2_conv
pool2_pool
conv3_block1_0_bn
conv3_block1_0_relu
conv3_block1_1_conv
co

In [ ]:
layer = 'predictions'

model = Model(inputs=base_model.input, outputs=base_model.get_layer(layer).output)


num_test = 50
X_train = np.concatenate((pos_images[num_test:], neg_images[num_test:]), axis=0)
X_test = np.concatenate((pos_images[:num_test], neg_images[:num_test]))

y_train = np.array([1] * (200 - num_test) + [0] * (200 - num_test))
y_test = np.array([1] * (num_test) + [0] * (num_test))

# X_train.shape, y_train.shape, X_test.shape, y_test.shape

# Define the model and train
from sklearn.linear_model import LogisticRegression

clfr = LogisticRegression()
clfr.fit(X_train, y_train)

# Evaluate
accuracy_score(clfr.predict(X_test), y_test)

0.87

## Lets try base model (ResNet50) and layer predictions

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
# from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:

base_model = ResNet50(weights='imagenet')

for layer in base_model.layers:
    print(layer.name)


layer = 'predictions'

model = Model(inputs=base_model.input, outputs=base_model.get_layer(layer).output)


num_test = 50
X_train = np.concatenate((pos_images[num_test:], neg_images[num_test:]), axis=0)
X_test = np.concatenate((pos_images[:num_test], neg_images[:num_test]))

y_train = np.array([1] * (200 - num_test) + [0] * (200 - num_test))
y_test = np.array([1] * (num_test) + [0] * (num_test))

# X_train.shape, y_train.shape, X_test.shape, y_test.shape

# Define the model and train
from sklearn.linear_model import LogisticRegression

clfr = LogisticRegression()
clfr.fit(X_train, y_train)

# Evaluate
accuracy_score(clfr.predict(X_test), y_test)

input_20
conv1_pad
conv1_conv
conv1_bn
conv1_relu
pool1_pad
pool1_pool
conv2_block1_1_conv
conv2_block1_1_bn
conv2_block1_1_relu
conv2_block1_2_conv
conv2_block1_2_bn
conv2_block1_2_relu
conv2_block1_0_conv
conv2_block1_3_conv
conv2_block1_0_bn
conv2_block1_3_bn
conv2_block1_add
conv2_block1_out
conv2_block2_1_conv
conv2_block2_1_bn
conv2_block2_1_relu
conv2_block2_2_conv
conv2_block2_2_bn
conv2_block2_2_relu
conv2_block2_3_conv
conv2_block2_3_bn
conv2_block2_add
conv2_block2_out
conv2_block3_1_conv
conv2_block3_1_bn
conv2_block3_1_relu
conv2_block3_2_conv
conv2_block3_2_bn
conv2_block3_2_relu
conv2_block3_3_conv
conv2_block3_3_bn
conv2_block3_add
conv2_block3_out
conv3_block1_1_conv
conv3_block1_1_bn
conv3_block1_1_relu
conv3_block1_2_conv
conv3_block1_2_bn
conv3_block1_2_relu
conv3_block1_0_conv
conv3_block1_3_conv
conv3_block1_0_bn
conv3_block1_3_bn
conv3_block1_add
conv3_block1_out
conv3_block2_1_conv
conv3_block2_1_bn
conv3_block2_1_relu
conv3_block2_2_conv
conv3_block2_2_bn
conv3

0.87

## Changing the base model to VGG19 Again

Using different output layer this time and MLP

In [ ]:
import numpy as np
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input

base_model = VGG19(weights='imagenet')

for layer in base_model.layers:
    print(layer.name)

input_21
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool
flatten
fc1
fc2
predictions


In [ ]:
layer = 'flatten'
model = Model(inputs=base_model.input, outputs=base_model.get_layer(layer).output)


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=25088))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))          
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=20,
          batch_size=10, verbose=False)
score = model.evaluate(X_test, y_test, batch_size=10)

10/10 [==============================] - 0s 12ms/step - loss: 0.3619 - accuracy: 0.8800


In [ ]:
score

[0.36189210414886475, 0.8799999952316284]

### Ques: Which model+layer works the best for this data? Why do you think that is?

I have tried different base models like VGG16, ResNet50, VGG19, DenseNet121 etc and layers like flaten and predictions. 

As per my experiment, I got similar accuracy (88%) for majority of the pre-trained model with different layer as the output from the base model. I have used 50% split for train and test data separation.
The probable reason behind getting better performance while using those layer as base model's output is - that layer might contain relatively more features that map to our intended binary classes and few of those features might have contributed to learn/identify the image class. 

### Ques: What makes for good positive examples? What makes for good negative examples? Why does the choice of negative examples matter?

I think the reason I had similar results despite changing layers and model is because of my choice of images. Because both the colours I chose are similar in nature they yielded similar results. The choice of negative examples definetly matters because while I used differnt images the reulst were very different and was varying based on the selection of model/layer

I think the if the examples are close enough to each other in both positive/negative case and very different from the opposites that would make them a very good example in both cases